# Оглавление

[1. Импорт модуля и загрузка датафрейма](#Импорт-модуля-и-загрузка-датафрейма)
<br>[2. Деление на тренировочную, валидационную и тестовую выборки](#Train,-test,-valid)
<br>[3. Лемматизция текста](#Лемматизация-текста)
<br>[4. Удаление стоп-слов](#Удаление-стоп-слов)
<br>[5. Преобразование в TF-IDF вектора](#Преобразование-в-TF-IDF-вектора)
<br>[6. Обучение модели логистической регрессии](#Обучение-модели-логистической-регрессии)
<br>[7. Запись модели в файл](#Запись-модели-в-файл)
<br>[8. Загрузка модели из файла](#Загрузка-модели-из-файла)
<br>[9. Предсказание эмоции для заданного текста](#Предсказание-эмоции-для-заданного-текста)

### Импорт модуля и загрузка датафрейма

In [2]:
import pickle
from tqdm import tqdm
from emotion_text import *

In [3]:
df = open_file('data.jsonl')

                                                text  label
0  i feel awful about it too because it s my job ...      0


In [4]:
#df_sampl = df.head(100).copy()

## Train, test, valid
Делим датасет на тренировочную, валидационную и тестовую выборки

In [5]:
X_train, y_train, X_test, X_val, y_test, y_val = train_test_val(df.text, df.label)

## Лемматизация текста

In [6]:
X_train, X_test, X_val = lemmatize_text(X_train, X_test, X_val)

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


169233    i once again fail to play it i ll doubtless st...
Name: text, dtype: object


## Удаление стоп-слов

In [7]:
X_train, X_test, X_val = stopwords_text(X_train, X_test, X_val, language = 'english', num_row=1)

169233    fail play doubtless still feel jealous
Name: text, dtype: object


## Преобразование в TF-IDF вектора

In [8]:
X_train_tfidf, X_test_tfidf, X_val_tfidf, vectorizer = tf_idf_vec(X_train, X_test, X_val)

## Обучение модели логистической регрессии

In [9]:
for C in tqdm(np.arange(0.1, 5.0, 0.1)):

    model_LogisticRegression_L2 = LogisticRegression(class_weight='balanced',
                                                     multi_class='multinomial',
                                                     max_iter = 1000,
                                                     solver='lbfgs',
                                                     penalty='l2',
                                                     C=C)
    learning_model(X_train_tfidf, y_train, X_val_tfidf, y_val, X_test_tfidf, y_test, model_LogisticRegression_L2, C)
print_score(max_f1_test, max_f1_valid, max_acc_test, max_acc_valid)

100%|██████████| 49/49 [1:44:37<00:00, 128.11s/it]

Максимальный f1_score = 0.8999273590179936, полученный при значении С = 0.9 на тестовой выборке
Максимальный f1_score = 0.9010047840724188, полученный при значении С = 0.8 на валидационной выборке
Максимальный accuracy = 0.8971113936805739, полученный при значении С = 0.9 на тестовой выборке
Максимальный accuracy = 0.8982629975288501, полученный при значении С = 0.8 на валидационной выборке


## Запись модели в файл

In [10]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model_LogisticRegression_L2, f)

## Загрузка модели из файла

In [11]:
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

## Предсказание эмоции для заданного текста

In [12]:
word = 'my day bad'

word_tfidf = vectorizer.transform([word])
vanga = model.predict(word_tfidf)

In [13]:
vanga_answers(vanga)

You are sadness, be happy!
